In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## IMPORT MODULES

In [69]:
from shutil import copyfile
copyfile(src = '../input/utils-py/Recommenders.py',dst='../working/Recommenders.py')

from Recommenders import *

In [70]:
import pandas as pd
import numpy as np
import Recommenders as Recommenders
import pickle


In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## LOADING THE DATASET

In [72]:
tfile=pd.read_csv('/kaggle/input/mock-data/triplets_file.csv')
tfile.head()

In [73]:
sfile=pd.read_csv('/kaggle/input/song-mock-data/song_data.csv')
sfile.head()

 Merging the tfile ans sfile in tsfile 

In [74]:
tsfile=pd.merge(tfile,sfile.drop_duplicates(['song_id']),on='song_id',how='left')
tsfile.head()

In [ ]:
##Length of tsfile(merged file)
len(tsfile)

In [76]:
print( len(tfile) ,len(sfile))

## DATA PREPROCESSING

Reducing size of a dataset

In [77]:
tsfile['song']=tsfile['title']+' - '+tsfile['artist_name']
tsfile=tsfile.head(10000)
tsfile.head()

In [78]:
tsfile['tags']=tsfile['song']+' - '+tsfile['release']+' - '+str(tsfile['year'])
tsfile.head()

In [99]:
##cummulative sum of listen count of the songs

ts_grouped=tsfile.groupby(['song']).agg({'listen_count':'count'}).reset_index()
ts_grouped.head()

In [80]:
ts_grouped.tail()

In [81]:
##sum of listen count in gsum
gsum=ts_grouped['listen_count'].sum()
## In percentage table
ts_grouped['percentage']=(ts_grouped['listen_count']/gsum)*100
ts_grouped.sort_values(['listen_count','song'],ascending=[0,1])

## POPULARITY RECOMMENDATION ENGINE

In [82]:
pr=Recommenders.popularity_recommender_py()

In [83]:
pr.create(tsfile,'user_id','song')

In [100]:
pr.recommend(tsfile['user_id'][9])

**This is popularity recommendation so all of them will be recommended same songs**

In [101]:
pr.recommend(tsfile['user_id'][100])

In [102]:
pr.recommend(tsfile['user_id'][500])

## ITEM SIMILARITY RECOMMENDATION

In [87]:
ir=Recommenders.item_similarity_recommender_py()

In [88]:
ir.create(tsfile,'user_id','song')
user_items=ir.get_user_items(tsfile['user_id'][10])

In [89]:
## Displaying user's song history
for i in user_items:
    print(i)

In [90]:
user_items=ir.get_user_items(tsfile['user_id'][100])

In [91]:
for i in user_items:
    print(i)

In [92]:
ir.recommend(tsfile['user_id'][10])

In [93]:
ir.recommend(tsfile['user_id'][100])

**Give related songs based on words**

In [94]:
ir.get_similar_items(['Love Story - Taylor Swift'])

In [95]:
ir.get_similar_items(['The Scientist - Coldplay'])

In [96]:
ir.get_similar_items(['The Scientist - Coldplay','Love Story - Taylor Swift'])

## USING PICKLE LIBRARY

In [97]:
pickle.dump(tsfile.to_dict(),open('Rrsongs.pkl','wb'))

In [103]:
pickle.dump(similarities,open('similarities.pkl','wb'))

In [ ]:
tsfile['song'].values